# シンプレックス法のアルゴリズムを実装する

In [1]:
import numpy as np
import scipy.linalg as linalg

In [2]:
MEPS = 1.0e-10

In [3]:
def lp_RevisedSimplex(c, A, b):
    np.seterr(divide='ignore') # 0割りなどの例外が生じてもエラーを飛ばさない
    (m, n) = A.shape
    
    # 初期化
    AI = np.hstack((A, np.identity(m))) # identity:正方単位行列を生成する
    c0 = np.r_[c, np.zeros(m)]
    basis = [n+i for i in range(m)] # 初期の基底変数のindex
    nonbasis = [j for j in range(n)]
    
    while True:
        # ## STEP1:最適性チェック
        # 双対変数の値を計算
        y = linalg.solve(AI[:, basis].T, c0[basis]) # linalg.solve : 連立方程式を解く
        cc = c0[nonbasis] - np.dot(y, AI[:, nonbasis]) # 被約費用ベクトル(reduced cost vector)
        # 実行可能解の判定
        if np.all(cc <= MEPS): # CC<=0であればyが双対問題の実行可能解
            # 最適基底解を出力して終了
            x = np.zeros(n+m)
            x[basis] = linalg.solve(AI[:, basis], b)
            print('Optimal')
            print('Optimal value = ', np.dot(c0[basis], x[basis]))
            for i in range(m):
                print('x', i, '=', x[i])
            break
        else:
            # 出る変数sを選択
            s = np.argmax(cc)
            
        # ## STEP2:非有界性チェック
        d = linalg.solve(AI[:,basis], AI[:, nonbasis[s]])
        # 非有界性判定
        if np.all(d <= MEPS):
            # 非有界
            print('Unbounded')
            break
        else:
            bb = linalg.solve(AI[:, basis], b)
            ratio = bb/d
            ratio[ratio<-MEPS] = np.inf
            r = np.argmin(ratio)
            # STEP3:基底と非基底の入れ替え
            nonbasis[s], basis[r] = basis[r], nonbasis[s]

In [4]:
# 例題
A = np.array([[2,2,-1], [2,-2,3], [0,2,-1]])
c = np.array([4,3,5])
b = np.array([6,8,4])

lp_RevisedSimplex(c, A, b)

Optimal
Optimal value =  45.0
x 0 = 0.0
x 1 = 4.999999999999999
x 2 = 6.0


# 内点法の実装
## 自己双対型線形最適化問題への変換

In [6]:
def make_Mq_from_cAb(c, A, b):
    """
    線形最適化問題を決定する行列A, ベクトルc,bを入力し、
    自己双対型線形最適化問題を決定する行列M,　ベクトルqを出力する
    """
    m, k = A.shape
    m1 = np.hstack((np.zeros((m,m)), -A, b.reshape(m, -1)))
    m2 = np.hstack((A.T, np.zeros((k,k)), -c.reshape(k, -1)))
    m3 = np.append(np.append(-b, c), 0)
    M = np.vstack((m1, m2, m3))
    q = np.zeros(m+k+1)
    return M, q

In [8]:
def make_artProb_initialPoint(M, q):
    """
    自己双対型線形最適化問題を決定する行列M,　ベクトルqから、
    それと等価な問題を決定する行列MM、ベクトルqq、初期内点を出力する
    """
    n,n = M.shape
    
    x0 = np.ones(n)
    mu0 = np.dot(q, x0)/(n+1)+1
    z0 = mu0/x0
    r = z0 - np.dot(M, x0) - q
    qn1 = (n+1)*mu0
    
    MM = np.hstack((M, r.reshape((-1,1))))
    MM = np.vstack((MM, np.append(-r, 0)))
    qq = np.append(q, qn1)
    xx0 = np.append(x0, 1)
    zz0 = np.append(z0, mu0)
    return MM, qq, xx0, zz0